In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import chromedriver_binary


In [ ]:
from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
chromedriver="C:\Python36\Scripts\chromedriver"

In [ ]:
driver = webdriver.Chrome(chromedriver)


# Flight Scraping

In [ ]:
url = 'https://www.transtats.bts.gov/ONTIME/Departures.aspx'

response = requests.get(url)

assert response.status_code == 200

In [ ]:
driver = webdriver.Chrome() 

In [ ]:
# This function automates the process of downloading csvs from 'https://www.transtats.bts.gov/ONTIME/Departures.aspx'
# by changing airport/airline fields as needed.

def get_stats(driver, airport_name, airline_name, month_number):
    
    driver.get(url)
    driver.find_element_by_tag_name('body').send_keys(Keys.HOME) 
    time.sleep(5)
    #soup = BeautifulSoup(driver.page_source, 'html.parser') 
    send_button = driver.find_element_by_xpath('//*[@id="chkAllStatistics"]') 
    send_button.click() 
    driver.execute_script("window.scrollTo(0, 150)")
    
    #time.sleep(15)
    airport = driver.find_element_by_xpath('//*[@id="cboAirport"]')
    airport.click() 
    airport.send_keys(airport_name) 
    
    #time.sleep(15)
    airline = driver.find_element_by_xpath('//*[@id="cboAirline"]') 
    airline.click() 
    airline.send_keys(airline_name) 
    
   # time.sleep(15)
    m_pat = '//*[@id="chkMonths_' + str(month_number) + '"]'
    month = driver.find_element_by_xpath(m_pat) 
    month.click() 
    
   # time.sleep(15)
    days = driver.find_element_by_xpath('//*[@id="chkAllDays"]')
    days.click() 
    
    driver.execute_script("window.scrollTo(150, 500)")
   # time.sleep(15)
    years = driver.find_element_by_xpath('//*[@id="chkYears_31"]')
    years.click() 

    submit = driver.find_element_by_xpath('//*[@id="btnSubmit"]')
    submit.click() 
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(10)
    
    download = driver.find_element_by_xpath('//*[@id="DL_CSV"]') 
    download.click() 
    

In [ ]:
city = 'San Francisco'

#air_name = 'United'

plane_lines = ['Southwest', 'Delta', 'United', 'American']
#plane_lines1 = ['Delta', 'United', 'American']
#plane_lines2 = ['Southwest', 'Delta', 'American']

month_nums = [0,3,5,9]


for num in month_nums:
    for p_name in plane_lines:
        get_stats(driver, city, p_name, num) 
   

In [ ]:
for i in range(1,144): 
    
    tag = 'EMPTY'
    
    if i <= 15:
        tag = 'SFO'
    elif i > 15 and i <= 31:
        tag = 'SEA'
    elif i > 31 and i <= 47:
        tag = 'PHX'
    elif i > 47 and i <= 55:
        tag = 'JFK'
    elif i > 55 and i <= 63:
        tag = 'LGA'
    elif i > 63 and i<= 75:
        tag = 'IAH'
    elif i > 75 and i <= 87:
        tag = 'HNL'
    elif i > 87 and i <= 103:
        tag = 'BNA'
    elif i > 103 and i <= 115:
        tag = 'STL'
    elif i > 115 and i <= 127:
        tag = 'CMH'
    elif i > 127 and i <= 143:
        tag = 'CLE'
        
    url = 'Detailed_Statistics_Departures (' + str(i) + ')' + '.csv'
    print(url)  
    new_df = pd.read_csv(url, skiprows=6)
    new_df['Origin'] = tag
    df = df.append(new_df)

In [ ]:
df = df[df['Carrier Code'] != ' SOURCE: Bureau of Transportation Statistics'] 
#df.to_pickle("flight_stats")

# Weather Scraping

In [ ]:
url = "https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-1"

response = requests.get(url)

assert response.status_code == 200

In [ ]:
driver = webdriver.Chrome() 

In [ ]:
def easy_scrape(driver, my_cols, month_dict, my_url, tag):
    #my_url = all_urls[0][1]
    #current_tag = all_urls[0][0]
    driver.get(my_url)
    time.sleep(10)
    tables = pd.read_html(driver.page_source)
    
    first_df = pd.concat([tables[3], tables[4], tables[5], tables[6], tables[7], tables[8]], axis=1)
    first_df = first_df.iloc[1:]
    first_df.columns = my_cols
    first_df.reset_index(inplace=True, drop=True)
    month = tables[2][0][0]
    prefix = month_dict[month]
    
    date_list = []

    to_iter = {'Jan' : 32, 'Apr' : 31, 'Jun' : 31, 'Oct' : 32}
    
    for i in range (1,to_iter[month]):
        date = prefix + '/' + str(i) + '/' + '2018'
        date_list.append(date)

    first_df['Date'] = date_list
    first_df['Origin'] = tag
    
    return first_df

In [ ]:
month_dict = {'Jan' : '01', 'Apr' : '04', 'Jun' : '06', 'Oct' : '10' }

In [ ]:
airport_urls = ['https://www.wunderground.com/history/monthly/us/ca/san-francisco/KSFO/date/2018-',
                'https://www.wunderground.com/history/monthly/us/wa/seattle/KSEA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/az/phoenix/KPHX/date/2018-',
                'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KJFK/date/2018-',
                'https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/tx/houston/KIAH/date/2018-',
                'https://www.wunderground.com/history/monthly/us/hi/honolulu/PHNL/date/2018-',
                'https://www.wunderground.com/history/monthly/us/tn/nashville/KBNA/date/2018-',
                'https://www.wunderground.com/history/monthly/us/mo/st.-louis/KSTL/date/2018-',
                'https://www.wunderground.com/history/monthly/us/oh/columbus/KCMH/date/2018-',              
               ] 

In [ ]:
main = pd.read_pickle('flight_stats')

In [ ]:
port_names = main['Origin'].unique() .tolist()
port_names = port_names[:-1]
port_names

In [ ]:
suffix = ['1', '4', '6', '10']
all_urls = []  
index = -1
for port in airport_urls:
    index += 1
    for s in suffix:
        final_url = port + s
        all_urls.append((port_names[index], final_url))
    

In [ ]:
all_urls[0:4]

In [ ]:
weather_df = pd.DataFrame()

In [ ]:
for link in all_urls:
    curr_df = easy_scrape(driver, my_cols, month_dict, link[1], link[0] )
    weather_df = weather_df.append(curr_df)
    

In [ ]:
#weather_df.to_pickle('with_rain')

In [ ]:
weather_df = pd.read_pickle('with_rain')

In [ ]:
new_cols = ['Origin', 'Date', 'Min_Temperature', 'Avg_Temperature', 'Max_Temperature', 'Min_DewPoint',
       'Avg_DewPoint', 'Max_DewPoint', 'Min_Humidity', 'Avg_Humidity',
       'Max_Humidity', 'Min_Wind', 'Avg_Wind', 'Max_Wind', 'Min_Pressure',
       'Avg_Pressure', 'Max_Pressure', 'Precipitation']

In [ ]:
weather_df = weather_df[new_cols]
weather_df.head()

In [ ]:
def change_date(date):
    if len(date) == 9:
        date = date[:2] + '/0' + date[3:]
    return date

In [ ]:
cp = weather_df.copy()
cp.reset_index(inplace=True,drop=True)
cp['Date'] = cp['Date'].apply(change_date)

In [ ]:
cp.head()

In [ ]:
#cp.to_pickle('with_rain_final')

# Combining Data

In [ ]:
flight_df = pd.read_pickle('flight_stats')
weather_df = pd.read_pickle('with_rain_final')

In [ ]:
flight_df.columns

In [ ]:
flight_df.head(5)

In [ ]:
weather_df.head(5)

In [ ]:
flight_df = flight_df.rename(columns={"Date (MM/DD/YYYY)" : "Date"})


In [ ]:
new_cols = ['Origin',
             'Date',
             'Carrier Code',
             'Flight Number',
             'Tail Number',
             'Destination Airport',
             'Scheduled departure time',
             'Actual departure time',
             'Scheduled elapsed time (Minutes)',
             'Actual elapsed time (Minutes)',
             'Departure delay (Minutes)',
             'Wheels-off time',
             'Taxi-Out time (Minutes)',
             'Delay Carrier (Minutes)',
             'Delay Weather (Minutes)',
             'Delay National Aviation System (Minutes)',
             'Delay Security (Minutes)',
             'Delay Late Aircraft Arrival (Minutes)',
             ]

In [ ]:
flight_df = flight_df[new_cols]

In [ ]:
edited = flight_df[flight_df['Origin'] != 'CLE']
mer = edited.merge(weather_df, on=['Origin', 'Date']) 


In [ ]:
mer['Origin'].unique()

In [ ]:
def get_gates(place):
    number_gates = {'SFO':115, 'SEA':80, 'PHX':116, 'JFK':128, 'LGA':72, 'IAH':130, 'HNL':47, 'BNA':42, 'STL':86, 'CMH':37}
    return number_gates[place]
    

In [ ]:
mer['Num Gates'] = mer['Origin'].apply(get_gates) 

In [ ]:
mer.sample(10) 

In [ ]:
#x = pd.Series(['Geeks', 'for', 'Geeks']) 

#unique_flights = pd.DataFrame(mer['Tail Number'].unique())

#def get_age(tail_num):
#    try:
#        url = 'https://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?NNumbertxt=' + str(tail_num)
#        tables = pd.read_html(url, match = 'A/W Date')
#        cert_date = tables[0][1][2]
#        return cert_date
#    except:
#        return 'NaN'
    
#unique_flights['Cert_Date'] = unique_flights['Tail Number'].apply(get_age)    

In [ ]:

mer = mer[mer['Tail Number'] != 'N389AA'] 

In [ ]:
check = pd.read_pickle('flight_ages')

check = check[check['Cert_Date'] != 'NaN']

final = mer.merge(check, on=['Tail Number'])

In [ ]:
final.columns



In [ ]:
dropped_cols = ['Scheduled elapsed time (Minutes)', 'Actual elapsed time (Minutes)', 'Flight Number']

final = final.drop(columns=dropped_cols)



In [ ]:
final.head()

In [ ]:
t_list = ['Origin', 'Date', 'Num Gates', 'Carrier Code', 'Tail Number', 'Cert_Date']
cols = final.columns.tolist()

new_cols = t_list + cols[4:-2]

final = final[new_cols] 

final = final[final['Cert_Date'] != 'None']

final.head()

In [ ]:
switch = final.copy()

switch = switch.rename(columns={"Min_Temperature" : "Max_Temp", "Max_Temperature" : "Min_Temp"})
switch = switch.rename(columns={"Min_DewPoint" : "Max_Dew", "Max_DewPoint" : "Min_Dew"})
switch = switch.rename(columns={"Min_Humidity" : "Max_Humid", "Max_Humidity" : "Min_Humid"})
switch = switch.rename(columns={"Min_Wind" : "Max_Wind", "Max_Wind" : "Min_Wind"})
switch = switch.rename(columns={"Min_Pressure" : "Max_Pressure", "Max_Pressure" : "Min_Pressure"})

switch.head()

In [ ]:
switch = switch.rename(columns={'Avg_Temperature' : 'Avg_Temp'})
#switch.to_pickle('FINAL3')